<a href="https://colab.research.google.com/github/william1209/gan/blob/main/cyclegan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Label Process

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip list | grep tensorflow

In [ ]:
import pandas as pd
import numpy as np
df_label = pd.read_table('/content/drive/My Drive/train_label.txt' )
df_label_test = pd.read_table('/content/drive/My Drive/test_label.txt')
df_label_desired = pd.read_table('/content/drive/My Drive/test_desired_age.txt')
print(len(df_label.index),len(df_label_test.index),len(df_label_desired.index))

In [ ]:
df_label_test.index

In [ ]:
def condition(df_label):
  #con0 = [df_label['age'][i]<=20 for i in range(len(df_label))]
  condition1 = [20<=df_label['age'][i]<36 for i in range(len(df_label))]
  condition2 = [36<=df_label['age'][i]<=52 for i in range(len(df_label))]
  condition3 = [70>=df_label['age'][i]>52 for i in range(len(df_label))]
  #con4 = [df_label['age'][i]>=52 for i in range(len(df_label))]
  return condition1,condition2,condition3

In [ ]:
def count(df_label):
  count1 = 0
  count2 = 0
  count3 = 0
  con1,con2,con3 = condition(df_label)
  for i in range(len(df_label)):
    if(con1[i]):
      count1+=1
    elif(con3[i]):
      count3+=1
    elif(con2[i]):
      count2+=1
  return count1,count2,count3

In [ ]:
def assign(df,t1,t2,t3):
  count1 = 0
  count2 = 0
  count3 = 0
  con1,con2,con3 = condition(df)
  index_list1 = np.zeros(t1)
  index_list2 = np.zeros(t2)
  index_list3 = np.zeros(t3)
  for i in range(len(df)):
    if(con1[i]):
      index_list1[count1] = i
      count1+=1
    elif(con3[i]):
      index_list3[count3] = i
      count3+=1
    elif(con2[i]):
      index_list2[count2] = i
      count2+=1
  return index_list1,index_list2,index_list3

In [ ]:
count1,count2,count3 = count(df_label)
print(count1,count2,count3)

In [ ]:
count1_test,count2_test,count3_test = count(df_label_test)
print(count1_test,count2_test,count3_test)

In [ ]:
index_c1,index_c2,index_c3 = assign(df_label,count1,count2,count3)

In [ ]:
index_c1_test,index_c2_test,index_c3_test = assign(df_label_test,count1_test,count2_test,count3_test)

In [ ]:
def to_df(df,index):
  pic = df['pic'][index]
  age = df['age'][index]
  dit = {'pic':pic,'age':age}
  df_to_return = pd.DataFrame(dit)
  return df_to_return

In [ ]:
c1 = to_df(df_label,index_c1)
c2 = to_df(df_label,index_c2)
c3 = to_df(df_label,index_c3)

c1_test = to_df(df_label_test,index_c1_test)
c2_test = to_df(df_label_test,index_c2_test)
c3_test = to_df(df_label_test,index_c3_test)


In [ ]:
c1.head()

In [ ]:
#training set init

c1_pic = c1['pic'].to_list()
c2_pic = c2['pic'].to_list()
c3_pic = c3['pic'].to_list()
c1_label = c1['age'].to_list()
c2_label = c2['age'].to_list()
c3_label = c3['age'].to_list()
c1_pic_path = []
c2_pic_path = []
c3_pic_path = []

In [ ]:
#test set init
c1_pic_test = c1_test['pic'].to_list()
c2_pic_test = c2_test['pic'].to_list()
c3_pic_test = c3_test['pic'].to_list()
c1_label_test = c1_test['age'].to_list()
c2_label_test = c2_test['age'].to_list()
c3_label_test = c3_test['age'].to_list()
c1_pic_path_test = []
c2_pic_path_test = []
c3_pic_path_test = []

In [ ]:
import os
import numpy as np

#file_dir = "/content/drive/My Drive/thumbnails128x128/train"
#c1_pic_path = []
for dirPath, dirNames, fileNames in os.walk(file_dir):
    #print(dirNames)
    for name in fileNames:
      if(name in c1_pic):
        c1_pic_path.append(os.path.join(dirPath, name))
        #print(os.path.basename(name),name)
      #elif(name in c2_pic):
      #  c2_pic_path.append(os.path.join(dirPath, name))
      #else:
      #  c3_pic_path.append(os.path.join(dirPath, name))

#print(len(c1_pic_path),len(c2_pic_path),len(c3_pic_path)) 
print(len(c1_pic_path))   

In [ ]:
#fetch train data

import glob
store = []
for num in range(65):
  num = num*1000
  num_str = str(num)
  num_str = num_str.zfill(5)
  addr = "/content/drive/My Drive/ffhq-dataset/thumbnails128x128/train/"+num_str+"/*.png"
  store.append(glob.iglob(addr))
                

In [ ]:
for i in range(len(store)):
  for ii in store[i]:
    print(os.path.basename(ii))

In [ ]:
for i in store[64]:
  #print(os.path.basename(i))
  if(os.path.basename(i) in c1_pic):
    c1_pic_path.append(i)
  
print(c1_pic_path)

In [ ]:
#parse training set path
import os
for ii in range(len(store)):
  for i in store[ii]:
    #print(os.path.basename(i))
    if(os.path.basename(i) in c1_pic):
      c1_pic_path.append(i)
    elif(os.path.basename(i) in c2_pic):
      c2_pic_path.append(i)
    elif(os.path.basename(i) in c3_pic):
      c3_pic_path.append(i)

print(len(c1_pic_path),len(c2_pic_path),len(c3_pic_path))


In [ ]:
from random import sample
c1_pic_path_sampled = sample(c1_pic_path,1000)
c2_pic_path_sampled = sample(c2_pic_path,1000)
c3_pic_path_sampled = sample(c3_pic_path,1000)
print(c1_pic_path_sampled)

In [ ]:
#fetch testdata

import glob
store_test = []
for num in range(65,70):
  num = num*1000
  num_str = str(num)
  num_str = num_str.zfill(5)
  addr = "/content/drive/My Drive/ffhq-dataset/thumbnails128x128/test/"+num_str+"/*.png"
  store_test.append(glob.iglob(addr))



In [ ]:
#parse test set path
for i in range(len(store_test)):
  for ii in store_test[i]:
    #print(os.path.basename(ii),ii)
    if(os.path.basename(ii) in c1_pic_test):
      c1_pic_path_test.append(ii)
    elif(os.path.basename(ii) in c2_pic_test):
      c2_pic_path_test.append(ii)
    elif(os.path.basename(ii) in c3_pic_test):
      c3_pic_path_test.append(ii)

print(len(c1_pic_path_test),len(c2_pic_path_test),len(c3_pic_path_test))

# Tansform to Tfrecord 

In [ ]:
import tensorflow as tf

# 二進位資料
def _bytes_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

# 整數資料
def _int64_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

# 浮點數資料
def _float32_feature(value):
  return tf.train.Feature(float_list=tf.train.FloatList(value=value))

In [ ]:
import cv2
def convert(images, labels, filename):
  #n_samples = len(labels)
  TFWriter = tf.compat.v1.python_io.TFRecordWriter(filename)

  print('\nTransform start...')
  for i in range(len(labels)):
    try:
      image = cv2.imread(images[i])
      #image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
      height,width,channel = image.shape
      #image = cv2.resize(image,(224,224))
            
      if image is None:
        print('Error image:' + images[i])
      else:
        image_raw = image.tostring()
        label = int(labels[i])
            
        # 將 tf.train.Feature 合併成 tf.train.Features
        ftrs = tf.train.Features(
              feature={'Label': _int64_feature(label),
                       'image_raw': _bytes_feature(image_raw),
                       'channel':_int64_feature(channel),
                       'width':_int64_feature(width),
                       'height':_int64_feature(height)}
                   )
        
        # 將 tf.train.Features 轉成 tf.train.Example
        example = tf.train.Example(features=ftrs)

        # 將 tf.train.Example 寫成 tfRecord 格式
        TFWriter.write(example.SerializeToString())
    except IOError as e:
      print('Skip!\n')

  TFWriter.close()
  print('Transform done!')

In [ ]:
convert(c3_pic_path,c3_label,'/content/drive/My Drive/thumbnails128x128/c3_train.tfrecords')

In [ ]:
tfrecord_path = '/root/tensorflow_datasets/cycle_gan/horse2zebra/2.0.0/cycle_gan-testA.tfrecord-00000-of-00001'

dataset_tf = tf.data.TFRecordDataset(tfrecord_path)

In [ ]:
for record in tf.compat.v1.python_io.tf_record_iterator(tfrecord_path):
    example = tf.train.Example()
    example.ParseFromString(record)

    img = example.features.feature['image_raw'].float_list.value
    label = example.features.feature['Label'].int64_list.value

In [ ]:
batch_size=32
tfrecord_dataset = tf.data.TFRecordDataset(tfrecord_path)
tfrecord_dataset = tfrecord_dataset.map(lambda   x:_parse_(x)).shuffle(True).batch(batch_size)

In [ ]:
tfrecord_dataset

# CycleGAN

This notebook demonstrates unpaired image to image translation using conditional GAN's, as described in [Unpaired Image-to-Image Translation using Cycle-Consistent Adversarial Networks](https://arxiv.org/abs/1703.10593), also known as CycleGAN. The paper proposes a method that can capture the characteristics of one image domain and figure out how these characteristics could be translated into another image domain, all in the absence of any paired training examples. 

This notebook assumes you are familiar with Pix2Pix, which you can learn about in the [Pix2Pix tutorial](https://www.tensorflow.org/tutorials/generative/pix2pix). The code for CycleGAN is similar, the main difference is an additional loss function, and the use of unpaired training data.

CycleGAN uses a cycle consistency loss to enable training without the need for paired data. In other words, it can translate from one domain to another without a one-to-one mapping between the source and target domain. 

This opens up the possibility to do a lot of interesting tasks like photo-enhancement, image colorization, style transfer, etc. All you need is the source and the target dataset (which is simply a directory of images).

![Output Image 1](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/generative/images/horse2zebra_1.png?raw=1)
![Output Image 2](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/generative/images/horse2zebra_2.png?raw=1)

## Set up the input pipeline

Install the [tensorflow_examples](https://github.com/tensorflow/examples) package that enables importing of the generator and the discriminator.

In [ ]:
!pip install git+https://github.com/tensorflow/examples.git

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow_examples.models.pix2pix import pix2pix

import os
import time
import matplotlib.pyplot as plt
from IPython.display import clear_output
import numpy as np

tfds.disable_progress_bar()
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
def solve_cudnn_error():
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)

## Input Pipeline

This tutorial trains a model to translate from images of horses, to images of zebras. You can find this dataset and similar ones [here](https://www.tensorflow.org/datasets/datasets#cycle_gan). 

As mentioned in the [paper](https://arxiv.org/abs/1703.10593), apply random jittering and mirroring to the training dataset. These are some of the image augmentation techniques that avoids overfitting.

This is similar to what was done in [pix2pix](https://www.tensorflow.org/tutorials/generative/pix2pix#load_the_dataset)

* In random jittering, the image is resized to `286 x 286` and then randomly cropped to `256 x 256`.
* In random mirroring, the image is randomly flipped horizontally i.e left to right.

In [ ]:
dataset = tfds.load('cycle_gan/horse2zebra',
                              with_info=False, as_supervised=True)
#metadata
ds_np = tfds.as_numpy(dataset)

In [ ]:
train_horses, train_zebras = dataset['trainA'], dataset['trainB']
test_horses, test_zebras = dataset['testA'], dataset['testB']

In [ ]:
(train_horses)
for i in train_horses:
  print(type(i))

In [ ]:
for i in train_horses:
  proto_tensor = tf.make_tensor_proto(i) 
  ll_train = tf.make_ndarray(proto_tensor)

In [ ]:

data_dirA = "/root/tensorflow_datasets/downloads/extracted/ZIP.peop.eecs.berk.edu_taes_park_Cycl_data_horp02dOS7syplmtRKHp2yhJq9HPZ3k8-j8DpzaCPoXMKE.zip/horse2zebra/trainA*/*"
data_dirB = "/root/tensorflow_datasets/downloads/extracted/ZIP.peop.eecs.berk.edu_taes_park_Cycl_data_horp02dOS7syplmtRKHp2yhJq9HPZ3k8-j8DpzaCPoXMKE.zip/horse2zebra/trainB*/*"
data_dirA_test = "/root/tensorflow_datasets/downloads/extracted/ZIP.peop.eecs.berk.edu_taes_park_Cycl_data_horp02dOS7syplmtRKHp2yhJq9HPZ3k8-j8DpzaCPoXMKE.zip/horse2zebra/testA*/*"
data_dirB_test = "/root/tensorflow_datasets/downloads/extracted/ZIP.peop.eecs.berk.edu_taes_park_Cycl_data_horp02dOS7syplmtRKHp2yhJq9HPZ3k8-j8DpzaCPoXMKE.zip/horse2zebra/testB*/*" 



list_ds_A = tf.compat.v1.data.Dataset.list_files(data_dirA)
list_ds_B = tf.compat.v1.data.Dataset.list_files(data_dirB)

list_ds_A_test = tf.compat.v1.data.Dataset.list_files(data_dirA_test)
list_ds_B_test = tf.compat.v1.data.Dataset.list_files(data_dirB_test)

In [ ]:
BUFFER_SIZE = 1000
BATCH_SIZE = 1
IMG_WIDTH = 256
IMG_HEIGHT = 256

In [ ]:
#temp1 = tf.compat.v1.data.Dataset.list_files(c1_pic_path)
temp1 = tf.compat.v1.data.Dataset.from_tensor_slices(c1_pic_path_sampled)
temp2 = tf.compat.v1.data.Dataset.from_tensor_slices(c2_pic_path_sampled)
temp3 = tf.compat.v1.data.Dataset.from_tensor_slices(c3_pic_path_sampled)

temp1_test = tf.compat.v1.data.Dataset.from_tensor_slices(c1_pic_path_test)
temp2_test = tf.compat.v1.data.Dataset.from_tensor_slices(c2_pic_path_test)
temp3_test = tf.compat.v1.data.Dataset.from_tensor_slices(c3_pic_path_test)

In [ ]:
print(type(temp3),type(temp3_test))

In [ ]:
def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
  return tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])

In [ ]:
def decode_img_png(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_png(img, channels=3)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
  return tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])

In [ ]:
def process_path(file_path):
  #label = get_label(file_path)

  # load the raw data from the file as a string
  #img = cv2.imread(file_path)
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img 

In [ ]:
def process_path_png(file_path):
  #label = get_label(file_path)

  # load the raw data from the file as a string
  #img = cv2.imread(file_path)
  img = tf.io.read_file(file_path)
  img = decode_img_png(img)
  return img 

In [ ]:
#mapping train data
labeled_temp1 = temp1.map(process_path_png, num_parallel_calls=AUTOTUNE).cache().shuffle(
    BUFFER_SIZE).batch(BATCH_SIZE)
labeled_temp2 = temp2.map(process_path_png, num_parallel_calls=AUTOTUNE).cache().shuffle(
    BUFFER_SIZE).batch(BATCH_SIZE)
labeled_temp3 = temp3.map(process_path_png, num_parallel_calls=AUTOTUNE).cache().shuffle(
    BUFFER_SIZE).batch(BATCH_SIZE)

#mapping test set

labeled_temp1_test = temp1_test.map(process_path_png, num_parallel_calls=AUTOTUNE).cache().shuffle(
    BUFFER_SIZE).batch(BATCH_SIZE)
labeled_temp2_test = temp2_test.map(process_path_png, num_parallel_calls=AUTOTUNE).cache().shuffle(
    BUFFER_SIZE).batch(BATCH_SIZE)
labeled_temp3_test = temp3_test.map(process_path_png, num_parallel_calls=AUTOTUNE).cache().shuffle(
    BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
sample_A = next(iter(labeled_temp1))
#sample_B = next(iter(labeled_temp2))
sample_C = next(iter(labeled_temp3))
#sample_A_test = 


In [ ]:
def random_crop(image):
  cropped_image = tf.image.random_crop(
      image, size=[IMG_HEIGHT, IMG_WIDTH, 3])

  return cropped_image

In [ ]:
# normalizing the images to [-1, 1]
def normalize(image):
  image = tf.cast(image, tf.float32)
  image = (image / 127.5) - 1
  return image

In [ ]:
def random_jitter(image):
  # resizing to 286 x 286 x 3
  image = tf.image.resize(image, [286, 286],
                          method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

  # randomly cropping to 256 x 256 x 3
  image = random_crop(image)

  # random mirroring
  image = tf.image.random_flip_left_right(image)

  return image

In [ ]:
def preprocess_image_train(image ,label):
  image = random_jitter(image)
  image = normalize(image)
  return image

In [ ]:
def preprocess_image_test(image ,label):
  image = normalize(image)
  return image

In [ ]:
plt.subplot(121)
plt.title('Young Bitch')
plt.imshow(sample_A[0] )

plt.subplot(122)
plt.title('Young Bitch with random jitter')
plt.imshow(random_jitter(sample_A[0]) )

In [ ]:
plt.subplot(121)
plt.title('Old Bitch')
plt.imshow(sample_C[0] )

plt.subplot(122)
plt.title('Old Bitch with random jitter')
plt.imshow(random_jitter(sample_C[0])  )

In [ ]:
sample_horse = sample_A
sample_zebra = sample_C

## Import and reuse the Pix2Pix models

Import the generator and the discriminator used in [Pix2Pix](https://github.com/tensorflow/examples/blob/master/tensorflow_examples/models/pix2pix/pix2pix.py) via the installed [tensorflow_examples](https://github.com/tensorflow/examples) package.

The model architecture used in this tutorial is very similar to what was used in [pix2pix](https://github.com/tensorflow/examples/blob/master/tensorflow_examples/models/pix2pix/pix2pix.py). Some of the differences are:

* Cyclegan uses [instance normalization](https://arxiv.org/abs/1607.08022) instead of [batch normalization](https://arxiv.org/abs/1502.03167).
* The [CycleGAN paper](https://arxiv.org/abs/1703.10593) uses a modified `resnet` based generator. This tutorial is using a modified `unet` generator for simplicity.

There are 2 generators (G and F) and 2 discriminators (X and Y) being trained here. 

* Generator `G` learns to transform image `X` to image `Y`. $(G: X -> Y)$
* Generator `F` learns to transform image `Y` to image `X`. $(F: Y -> X)$
* Discriminator `D_X` learns to differentiate between image `X` and generated image `X` (`F(Y)`).
* Discriminator `D_Y` learns to differentiate between image `Y` and generated image `Y` (`G(X)`).

![Cyclegan model](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/generative/images/cyclegan_model.png?raw=1)

In [ ]:
OUTPUT_CHANNELS = 3

generator_g = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')
generator_f = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')

discriminator_x = pix2pix.discriminator(norm_type='instancenorm', target=False)
discriminator_y = pix2pix.discriminator(norm_type='instancenorm', target=False)

In [ ]:
to_zebra = generator_g(sample_horse)
to_horse = generator_f(sample_zebra)
plt.figure(figsize=(8, 8))
contrast = 10

imgs = [sample_horse, to_zebra, sample_zebra, to_horse]
title = ['Horse', 'To Zebra', 'Zebra', 'To Horse']

for i in range(len(imgs)):
  plt.subplot(2, 2, i+1)
  plt.title(title[i])
  if i % 2 == 0:
    plt.imshow(imgs[i][0]  )
    #plt.imshow(imgs[i][0] )
  else:
    plt.imshow(imgs[i][0]  * contrast )
    #plt.imshow(imgs[i][0] )
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))

plt.subplot(121)
plt.title('Is a real zebra?')
plt.imshow(discriminator_y(sample_zebra)[0, ..., -1], cmap='RdBu_r')

plt.subplot(122)
plt.title('Is a real horse?')
plt.imshow(discriminator_x(sample_horse)[0, ..., -1], cmap='RdBu_r')

plt.show()

## Loss functions

In CycleGAN, there is no paired data to train on, hence there is no guarantee that the input `x` and the target `y` pair are meaningful during training. Thus in order to enforce that the network learns the correct mapping, the authors propose the cycle consistency loss.

The discriminator loss and the generator loss are similar to the ones used in [pix2pix](https://www.tensorflow.org/tutorials/generative/pix2pix#define_the_loss_functions_and_the_optimizer).

In [ ]:
LAMBDA = 10

In [ ]:
loss_obj = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def discriminator_loss(real, generated):
  real_loss = loss_obj(tf.ones_like(real), real)

  generated_loss = loss_obj(tf.zeros_like(generated), generated)

  total_disc_loss = real_loss + generated_loss

  return total_disc_loss * 0.5

In [ ]:
def generator_loss(generated):
  return loss_obj(tf.ones_like(generated), generated)

Cycle consistency means the result should be close to the original input. For example, if one translates a sentence from English to French, and then translates it back from French to English, then the resulting sentence should be the same as the  original sentence.

In cycle consistency loss, 

* Image $X$ is passed via generator $G$ that yields generated image $\hat{Y}$.
* Generated image $\hat{Y}$ is passed via generator $F$ that yields cycled image $\hat{X}$.
* Mean absolute error is calculated between $X$ and $\hat{X}$.

$$forward\ cycle\ consistency\ loss: X -> G(X) -> F(G(X)) \sim \hat{X}$$

$$backward\ cycle\ consistency\ loss: Y -> F(Y) -> G(F(Y)) \sim \hat{Y}$$


![Cycle loss](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/generative/images/cycle_loss.png?raw=1)

In [ ]:
def calc_cycle_loss(real_image, cycled_image):
  loss1 = tf.reduce_mean(tf.abs(real_image - cycled_image))
  
  return LAMBDA * loss1

As shown above, generator $G$ is responsible for translating image $X$ to image $Y$. Identity loss says that, if you fed image $Y$ to generator $G$, it should yield the real image $Y$ or something close to image $Y$.

$$Identity\ loss = |G(Y) - Y| + |F(X) - X|$$

In [ ]:
def identity_loss(real_image, same_image):
  loss = tf.reduce_mean(tf.abs(real_image - same_image))
  return LAMBDA * 0.5 * loss

Initialize the optimizers for all the generators and the discriminators.

In [ ]:
generator_g_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
generator_f_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

discriminator_x_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_y_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

## Checkpoints

In [ ]:
#checkpoint_path = "./checkpoints/train"
checkpoint_path = "/content/drive/My Drive/checkpointA2C/"

ckpt = tf.train.Checkpoint(generator_g=generator_g,
                           generator_f=generator_f,
                           discriminator_x=discriminator_x,
                           discriminator_y=discriminator_y,
                           generator_g_optimizer=generator_g_optimizer,
                           generator_f_optimizer=generator_f_optimizer,
                           discriminator_x_optimizer=discriminator_x_optimizer,
                           discriminator_y_optimizer=discriminator_y_optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print ('Latest checkpoint restored!!')

## Training

Note: This example model is trained for fewer epochs (40) than the paper (200) to keep training time reasonable for this tutorial. Predictions may be less accurate. 

In [ ]:
EPOCHS = 128

In [ ]:
def generate_images(model, test_input):
  prediction = model(test_input)
    
  plt.figure(figsize=(12, 12))

  display_list = [test_input[0], prediction[0]]
  title = ['Input Image', 'Predicted Image']

  for i in range(2):
    plt.subplot(1, 2, i+1)
    plt.title(title[i])
    # getting the pixel values between [0, 1] to plot it.
    plt.imshow(display_list[i])
    plt.axis('off')
  plt.show()

Even though the training loop looks complicated, it consists of four basic steps:

* Get the predictions.
* Calculate the loss.
* Calculate the gradients using backpropagation.
* Apply the gradients to the optimizer.

In [ ]:
@tf.function
def train_step(real_x, real_y):
  # persistent is set to True because the tape is used more than
  # once to calculate the gradients.
  with tf.GradientTape(persistent=True) as tape:
    # Generator G translates X -> Y
    # Generator F translates Y -> X.
    
    fake_y = generator_g(real_x, training=True)
    cycled_x = generator_f(fake_y, training=True)

    fake_x = generator_f(real_y, training=True)
    cycled_y = generator_g(fake_x, training=True)

    # same_x and same_y are used for identity loss.
    same_x = generator_f(real_x, training=True)
    same_y = generator_g(real_y, training=True)

    disc_real_x = discriminator_x(real_x, training=True)
    disc_real_y = discriminator_y(real_y, training=True)

    disc_fake_x = discriminator_x(fake_x, training=True)
    disc_fake_y = discriminator_y(fake_y, training=True)

    # calculate the loss
    gen_g_loss = generator_loss(disc_fake_y)
    gen_f_loss = generator_loss(disc_fake_x)
    
    total_cycle_loss = calc_cycle_loss(real_x, cycled_x) + calc_cycle_loss(real_y, cycled_y)
    
    # Total generator loss = adversarial loss + cycle loss
    total_gen_g_loss = gen_g_loss + total_cycle_loss + identity_loss(real_y, same_y)
    total_gen_f_loss = gen_f_loss + total_cycle_loss + identity_loss(real_x, same_x)

    disc_x_loss = discriminator_loss(disc_real_x, disc_fake_x)
    disc_y_loss = discriminator_loss(disc_real_y, disc_fake_y)
  
  # Calculate the gradients for generator and discriminator
  generator_g_gradients = tape.gradient(total_gen_g_loss, 
                                        generator_g.trainable_variables)
  generator_f_gradients = tape.gradient(total_gen_f_loss, 
                                        generator_f.trainable_variables)
  
  discriminator_x_gradients = tape.gradient(disc_x_loss, 
                                            discriminator_x.trainable_variables)
  discriminator_y_gradients = tape.gradient(disc_y_loss, 
                                            discriminator_y.trainable_variables)
  
  # Apply the gradients to the optimizer
  generator_g_optimizer.apply_gradients(zip(generator_g_gradients, 
                                            generator_g.trainable_variables))

  generator_f_optimizer.apply_gradients(zip(generator_f_gradients, 
                                            generator_f.trainable_variables))
  
  discriminator_x_optimizer.apply_gradients(zip(discriminator_x_gradients,
                                                discriminator_x.trainable_variables))
  
  discriminator_y_optimizer.apply_gradients(zip(discriminator_y_gradients,
                                                discriminator_y.trainable_variables))

In [ ]:
#train_horses = labeled_ds_A
#train_zebras = labeled_ds_B

train_horses = labeled_temp1
train_zebras = labeled_temp3

In [ ]:
for epoch in range(EPOCHS):
  start = time.time()

  n = 0
  for image_x, image_y in tf.data.Dataset.zip((train_horses, train_zebras)):
    train_step(image_x, image_y)
    if n % 10 == 0:
      print ('.', end='')
    n+=1

  clear_output(wait=True)
  # Using a consistent image (sample_horse) so that the progress of the model
  # is clearly visible.
  generate_images(generator_g, sample_horse)

  if (epoch + 1) % 5 == 0:
    ckpt_save_path = ckpt_manager.save()
    print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                         ckpt_save_path))

  print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                      time.time()-start))

## Generate using test dataset

In [ ]:
# Run the trained model on the test dataset
for inp in labeled_temp1_test:
  generate_images(generator_g, inp)
  
  #generate_images(generator_f,inp)

In [ ]:
len(labeled_temp1_test)

In [ ]:
# Run the trained model on the test dataset
for inp in labeled_temp1.take(5):
  #generate_images(generator_g, inp)
  generate_images(generator_f,inp)

###分隔線

In [ ]:
test_compare = df_label_test['age'].to_list()
desire_compare = df_label_desired['age'].to_list()
test_compare_path = df_label_test['pic'].to_list()
test_entire_path = []

In [ ]:
#fetch testdata

import glob
store_test = []
for num in range(65,70):
  num = num*1000
  num_str = str(num)
  num_str = num_str.zfill(5)
  addr = "/content/drive/My Drive/ffhq-dataset/thumbnails128x128/test/"+num_str+"/*.png"
  store_test.append(glob.iglob(addr))


#parse test set path
for i in range(len(store_test)):
  for ii in store_test[i]:
    #print(os.path.basename(ii),ii)
    test_entire_path.append(ii)


print(len(test_entire_path))

In [ ]:
test_entire_path = tf.compat.v1.data.Dataset.from_tensor_slices(test_entire_path)
test_entire_path_processed = test_entire_path.map(process_path_png, num_parallel_calls=AUTOTUNE).cache().shuffle(
    BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:

#test_compare = df_label_test['age'].to_list()
#desire_compare = df_label_desired['age'].to_list()
#test_entire_path = tf.compat.v1.data.Dataset.from_tensor_slices(test_entire_path)
#test_entire_path_processed = test_entire_path.map(process_path_png, num_parallel_calls=AUTOTUNE).cache().shuffle(
#    BUFFER_SIZE).batch(BATCH_SIZE)
aged = []
rec = []

count_4_iter = 0
for i in test_entire_path_processed:
  if(test_compare[count_4_iter]<desire_compare[count_4_iter]):
    aged.append(generator_f(i)) #young2old
    rec.append(generator_g(i)) #old back to young
  else:
    aged.append(generator_g(i)) #old2young
    rec.append(generator_f(i)) #young back to old
